In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

# Get the Data

In [2]:
#load the dataset
retail_df = pd.read_csv('data.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


# Prepare the Data


As customer clusters may vary by geography, I’ll restrict the data to only United Kingdom customers, which contains most of our customers historical data.

In [3]:
retail_uk = retail_df[retail_df['Country']=='United Kingdom']
#check the shape
retail_uk.shape

(495478, 8)

In [4]:
#remove canceled orders
retail_uk = retail_uk[retail_uk['Quantity']>0]
retail_uk.shape

(486286, 8)

In [5]:
#remove rows where customerID are NA
retail_uk.dropna(subset=['CustomerID'],how='all',inplace=True)
retail_uk.shape

(354345, 8)

In [6]:
#restrict the data to one full year because it's better to use a metric per Months or Years in RFM
retail_uk = retail_uk[retail_uk['InvoiceDate']>= "2010-12-09"]
retail_uk.shape

(176137, 8)

In [7]:
print("Summary..")
#exploring the unique values of each attribute
print("Number of transactions: ", retail_uk['InvoiceNo'].nunique())
print("Number of products bought: ",retail_uk['StockCode'].nunique())
print("Number of customers:", retail_uk['CustomerID'].nunique() )
print("Percentage of customers NA: ", round(retail_uk['CustomerID'].isnull().sum() * 100 / len(retail_df),2),"%" )

Summary..
Number of transactions:  8789
Number of products bought:  3294
Number of customers: 2864
Percentage of customers NA:  0.0 %


# RFM Analysis

**RFM** (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. 
RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

- RECENCY (R): Days since last purchase
- FREQUENCY (F): Total number of purchases
- MONETARY VALUE (M): Total money this customer spent.

We will create those 3 customer attributes for each customer.

## Recency

To calculate recency, we need to choose a date point from which we evaluate **how many days ago was the customer's last purchase**.

In [8]:
#last date available in our dataset
retail_uk['InvoiceDate'].max()

'9/9/2011 9:52'

The last date we have is 2011-12-09 so we will use it as reference.

In [9]:
now = dt.date(2011,12,9)
print(now)

2011-12-09


In [10]:
#create a new column called date which contains the date of invoice only
retail_uk['date'] = pd.DatetimeIndex(retail_uk['InvoiceDate']).date

In [11]:
retail_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
105335,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,3/1/2011 8:30,7.95,14620,United Kingdom,2011-03-01
105336,545220,48194,DOORMAT HEARTS,2,3/1/2011 8:30,7.95,14620,United Kingdom,2011-03-01
105337,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,3/1/2011 8:30,1.65,14620,United Kingdom,2011-03-01
105338,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,3/1/2011 8:30,4.95,14620,United Kingdom,2011-03-01
105339,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3/1/2011 8:30,3.75,14620,United Kingdom,2011-03-01


In [12]:
#group by customers and check last date of purshace
recency_df = retail_uk.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurshaceDate']
recency_df.head()

,CustomerID,LastPurshaceDate
0,12747,2011-08-22
1,12748,2011-09-30
2,12749,2011-08-01
3,12820,2011-09-26
4,12821,2011-05-09


In [13]:

#calculate recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [14]:
recency_df.head()

,CustomerID,LastPurshaceDate,Recency
0,12747,2011-08-22,109
1,12748,2011-09-30,70
2,12749,2011-08-01,130
3,12820,2011-09-26,74
4,12821,2011-05-09,214


In [15]:
#drop LastPurchaseDate as we don't need it anymore
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)

## Frequency

Frequency helps us to know how many times a customer purchased from us. To do that we need to check **how many invoices are registered by the same customer**.

In [16]:
# drop duplicates
retail_uk_copy = retail_uk
retail_uk_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_df = retail_uk_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

,CustomerID,Frequency
0,12747,5
1,12748,96
2,12749,3
3,12820,1
4,12821,1


## Monetary

Monetary attribute answers the question: **How much money did the customer spent over time?**

To do that, first, we will create a new column total cost to have the total price per invoice.

In [17]:
#create column total cost
retail_uk['TotalCost'] = retail_uk['Quantity'] * retail_uk['UnitPrice']

In [18]:
monetary_df = retail_uk.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

,CustomerID,Monetary
0,12747,191.85
1,12748,1054.43
2,12749,67.00
3,12820,15.00
4,12821,19.92


## Create RFM Table

In [19]:
#merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='CustomerID')
temp_df.head()

,CustomerID,Recency,Frequency
0,12747,109,5
1,12748,70,96
2,12749,130,3
3,12820,74,1
4,12821,214,1


In [20]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='CustomerID')
#use CustomerID as index
rfm_df.set_index('CustomerID',inplace=True)
#check the head
rfm_df.head()
temp = rfm_df.copy()

## RFM Table Correctness verification

In [21]:
retail_uk[retail_uk['CustomerID']=='12820']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,TotalCost
360567,568236,23328,SET 6 SCHOOL MILK BOTTLES IN CRATE,4,9/26/2011 11:49,3.75,12820,United Kingdom,2011-09-26,15.0


In [22]:
(now - dt.date(2011,9,26)).days == 74

True

## Customer segments with RFM Model

The simplest way to create customers segments from RFM Model is to use** Quartiles**. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

Note: Quintiles (score from 1-5) offer better granularity, in case the business needs that but it will be more challenging to create segments since we will have 555 possible combinations. So, we will use quartiles.

### RFM Quartiles

In [23]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,85.0,1.0,16.35
0.50,119.0,2.0,35.40
0.75,183.0,3.0,92.42


In [24]:
quantiles.to_dict()

{'Recency': {0.25: 85.0, 0.5: 119.0, 0.75: 183.0},
 'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 3.0},
 'Monetary': {0.25: 16.35, 0.5: 35.400000000000006, 0.75: 92.42}}

### Creation of RFM Segments

We will create two segmentation classes since, high recency is bad, while high frequency and monetary value is good.

In [25]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [26]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [27]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
CustomerID,,,,,,
12747,109,5,191.85,3,4,4
12748,70,96,1054.43,4,4,4
12749,130,3,67.00,2,3,3
12820,74,1,15.00,4,1,1
12821,214,1,19.92,1,1,2


Now that we have the score of each customer, we can represent our customer segmentation.
First, we need to combine the scores (R_Quartile, F_Quartile,M_Quartile) together.

In [28]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12747,109,5,191.85,3,4,4,344
12748,70,96,1054.43,4,4,4,444
12749,130,3,67.00,2,3,3,233
12820,74,1,15.00,4,1,1,411
12821,214,1,19.92,1,1,2,112


Best Recency score = 4: most recently purchase. Best Frequency score = 4: most quantity purchase. 
Best Monetary score = 4: spent the most.

Let's see who are our** Champions** (best customers).

In [29]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
18102,72,34,26632.62,4,4,4,444
17949,70,32,22504.73,4,4,4,444
17450,70,28,18009.06,4,4,4,444
16029,80,39,15119.49,4,4,4,444
16013,70,24,10402.34,4,4,4,444
12901,81,20,5915.66,4,4,4,444
13798,72,34,4648.80,4,4,4,444
17857,72,12,4644.68,4,4,4,444
13694,71,32,4472.68,4,4,4,444


We can find [here](http://www.blastam.com/blog/rfm-analysis-boosts-sales) a suggestion of key segments and then we can decide which segment to consider for further study.

Note: the suggested link use the opposite valuation: 1 as highest/best score and 4 is the lowest.

**How many customers do we have in each segment?**

In [30]:
# print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
# print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
# print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
# print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
# print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
# print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))


data = {'Metrics': ['Best Customers', 'Loyal Customers', 'Big Spenders', 'Almost Lost', 'Lost Customers', 'Lost Cheap Customers'],
        'Count': [len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']),
                  len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]),
                  len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]),
                  len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']),
                  len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']),
                  len(rfm_segmentation[rfm_segmentation['RFMScore']=='111'])]}

df = pd.DataFrame(data)
df.set_index('Metrics',inplace=True)
print(df)
df.to_csv('segments.csv')

                      Count
Metrics                    
Best Customers          218
Loyal Customers         687
Big Spenders            716
Almost Lost              52
Lost Customers            5
Lost Cheap Customers    278


Now that we knew our customers segments we can choose how to target or deal with each segment.

For example:

**Best Customers - Champions**: Reward them. They can be early adopters to new products. Suggest them "Refer a friend".

**At Risk**: Send them personalized emails to encourage them to shop.

More ideas about what actions to perform in [Ometria](http://54.73.114.30/customer-segmentation#).

In [31]:
import numpy as np

conditions = [
 (rfm_segmentation['R_Quartile'] >= 4) & (rfm_segmentation['F_Quartile'] >= 4) & (rfm_segmentation['M_Quartile'] >= 4), # 'Best Customers'
 (rfm_segmentation['R_Quartile'] < 4) & (rfm_segmentation['F_Quartile'] >= 4) & (rfm_segmentation['M_Quartile'] >= 4), # 'Loyal Customers'
 (rfm_segmentation['R_Quartile'] < 4) & (rfm_segmentation['F_Quartile'] < 4) & (rfm_segmentation['M_Quartile'] >= 4), # 'Big Spenders'
 (rfm_segmentation['R_Quartile'] > 3) & (rfm_segmentation['F_Quartile'] < 4) & (rfm_segmentation['M_Quartile'] < 4), # 'Almost Lost'
 (rfm_segmentation['R_Quartile'] < 4) & (rfm_segmentation['F_Quartile'] < 4) & (rfm_segmentation['M_Quartile'] < 4), # 'Lost Customers'
]

labels = ['Best Customers', 'Loyal Customers', 'Big Spenders', 'Almost Lost', 'Lost Customers']

# Assign the labels to the RFM values
rfm_segmentation['CustomerSegment'] = np.select(conditions, labels, default='Other')


In [32]:
rfm_segmentation.to_csv('rfm_segmentation.csv')

In [33]:
rfm_segmentation['CustomerSegment'].unique()

array(['Loyal Customers', 'Best Customers', 'Lost Customers',
       'Almost Lost', 'Other', 'Big Spenders'], dtype=object)

In [34]:
temp['CustomerSegment'] = rfm_segmentation['CustomerSegment']

In [35]:
temp

,Recency,Frequency,Monetary,CustomerSegment
CustomerID,,,,
12747,109,5,191.85,Loyal Customers
12748,70,96,1054.43,Best Customers
12749,130,3,67.00,Lost Customers
12820,74,1,15.00,Almost Lost
12821,214,1,19.92,Lost Customers
...,...,...,...,...
18280,277,1,23.70,Lost Customers
18281,180,1,5.04,Lost Customers
18282,126,1,12.75,Lost Customers


In [36]:
temp.to_csv('finalData.csv')

In [37]:
temp['CustomerSegment'].unique()

array(['Loyal Customers', 'Best Customers', 'Lost Customers',
       'Almost Lost', 'Other', 'Big Spenders'], dtype=object)

## Creating AN ANN


In [38]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and it includes the column 'CustomerSegment'
le = LabelEncoder()
temp['CustomerSegment'] = le.fit_transform(temp['CustomerSegment'])


In [39]:
from sklearn.model_selection import train_test_split

X = temp.drop('CustomerSegment', axis=1) # Features
y = temp['CustomerSegment'] # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from keras.utils import to_categorical

# One-hot encode the target variable
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)



In [40]:
y_train.unique()

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [41]:
from keras.models import Sequential
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(X_train.shape[1],)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=150, batch_size=10)

loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))


Epoch 1/150
230/230 [==============================] - 2s 3ms/step - loss: 26.4381 - accuracy: 0.0262
Epoch 2/150
230/230 [==============================] - 1s 4ms/step - loss: 3.7924 - accuracy: 0.4609
Epoch 3/150
230/230 [==============================] - 1s 4ms/step - loss: 1.6429 - accuracy: 0.6268
Epoch 4/150
230/230 [==============================] - 1s 4ms/step - loss: 1.3743 - accuracy: 0.6530
Epoch 5/150
230/230 [==============================] - 1s 3ms/step - loss: 2.6777 - accuracy: 0.6504
Epoch 6/150
230/230 [==============================] - 1s 3ms/step - loss: 1.1836 - accuracy: 0.6643
Epoch 7/150
230/230 [==============================] - 1s 4ms/step - loss: 1.6952 - accuracy: 0.6569
Epoch 8/150
230/230 [==============================] - 1s 3ms/step - loss: 1.4522 - accuracy: 0.6648
Epoch 9/150
230/230 [==============================] - 1s 3ms/step - loss: 0.9797 - accuracy: 0.6801
Epoch 10/150
230/230 [==============================] - 1s 3ms/step - loss: 2.0940 - accur

In [ ]:
# Generate predictions for the test data
predictions = model.predict(X_test)
predictions

18/18 [==============================] - 0s 5ms/step


array([[8.48637439e-16, 6.03238177e-16, 1.51782657e-03, 9.93354321e-01,
        7.09179221e-05, 5.05683199e-03],
       [8.58955245e-05, 3.68134620e-11, 1.12741945e-04, 9.99691367e-01,
        1.48436030e-07, 1.09886256e-04],
       [2.54962765e-06, 5.63867157e-04, 9.15589392e-01, 8.94599408e-03,
        4.82141040e-02, 2.66840756e-02],
       ...,
       [1.15622190e-09, 7.05876353e-21, 1.96463816e-07, 9.99999642e-01,
        3.08132995e-13, 6.00761183e-08],
       [1.61367529e-21, 9.45746541e-01, 1.05984952e-06, 1.43309058e-17,
        5.40156141e-02, 2.36747874e-04],
       [0.00000000e+00, 9.68919337e-01, 1.78014714e-09, 0.00000000e+00,
        3.10804658e-02, 2.18157922e-07]], dtype=float32)